Rachel Rathgeber
DA320
Midterm

In [9]:
import re
import urllib3
import certifi

#constract all regular expressions
title_regex=re.compile(r"class=\"title\"><h3>(.+)</h3>")
date_regex=re.compile(r"class=\"clamp-details\">\s+<span>(.+)</span>")
description_regex=re.compile(r"<div class=\"summary\">\s*([\S\s]+?)\s*</div>")
score_regex = re.compile(r"<span class=\"title\">Metascore:</span>\s+<a class=\"metascore_anchor\" href=\"/movie/.*?/critic-reviews\">\s+<div class=\"metascore_w large movie .+\">(.*?)</div>")
image_regex = re.compile(r"<a href=\"/movie/.*\"><img src=\"(.*)\" alt=\"")

# Construct an HTTP pool for connection
http=urllib3.PoolManager(ca_certs=certifi.where())




In [10]:
import json
import pymongo
import certifi

#retrieve credentials for sql server from credentials file
with open('/Users/rachelrathgeber/Documents/DA320/credentials.json') as f:
    data = json.load(f)
    mongo_connection_string = data['mongodb']

#fetch the database
client = pymongo.MongoClient(mongo_connection_string, tlsCAFile=certifi.where())
da320_database = client['DA320']
metacritic_data = da320_database['metacritic']

In [11]:
import pandas



#retrieve a list of movies from a particular year and page of metacritic
def metacritic_scraper(year: int, page: int) -> pandas.DataFrame:

    #fetch the metacritic webpage
    url = f"https://www.metacritic.com/browse/movies/score/metascore/year/filtered?year_selected={year}&sort=desc&view=detailed&page={page}"
    response = http.request('GET', url, headers={'User-Agent':'Mozilla/5.0'})
    datastring = str(response.data, "utf-8")

    #execute the regular expressions
    titles = title_regex.findall(datastring)
    dates = date_regex.findall(datastring)
    description = description_regex.findall(datastring)
    scores = score_regex.findall(datastring)
    images = image_regex.findall(datastring)

    #return a unified collection
    dataset = { "title": titles, "date": dates, "description": description, "score": scores, "image": images}
    return pandas.DataFrame(dataset)

In [13]:
import re
import time

#write a CSV file for this data
for year in range(2000, 2023):
    page = 0
    print(f"Collecting data for {year} page {page}...")

    #retry a page multiple times if necessary
    while True:
        data = metacritic_scraper(year, page)

        #stop when we reach a page with zero rows
        if len(data) == 0:
            break

        #convert the dataframe into a list of movies to insert into MongoDB
        movies_to_insert = []
        for row in data.itertuples():
            movie = {
                "title": row.title,
                "release_date": row.date,
                "description": row.description,
                "metascore": row.score,
                "imgage_url": row.image,
            }
            movies_to_insert.append(movie)

        #insert records into MongoDB
        print(f"Inserting {len(movies_to_insert)} movies for the year {year} page {page}")
        metacritic_data.insert_many(movies_to_insert)
        page = page + 1

Inserting 100 movies for the year 2000 page 0
Inserting 100 movies for the year 2000 page 1
Inserting 100 movies for the year 2000 page 2
Inserting 65 movies for the year 2000 page 3
Inserting 100 movies for the year 2001 page 0
Inserting 100 movies for the year 2001 page 1
Inserting 100 movies for the year 2001 page 2
Inserting 85 movies for the year 2001 page 3
Inserting 100 movies for the year 2002 page 0
Inserting 100 movies for the year 2002 page 1
Inserting 100 movies for the year 2002 page 2
Inserting 100 movies for the year 2002 page 3
Inserting 30 movies for the year 2002 page 4
Inserting 100 movies for the year 2003 page 0
Inserting 100 movies for the year 2003 page 1
Inserting 100 movies for the year 2003 page 2
Inserting 100 movies for the year 2003 page 3
Inserting 9 movies for the year 2003 page 4
Inserting 100 movies for the year 2004 page 0
Inserting 100 movies for the year 2004 page 1
Inserting 100 movies for the year 2004 page 2
Inserting 100 movies for the year 2004 